<a href="https://colab.research.google.com/github/KakarotDB/SLMCohort/blob/main/ImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#Build an Image Classifier on MNIST
#    - Use torchvision.datasets to load MNIST.
#    - Define a neural network with nn.Module.
#    - Train and evaluate the model’s accuracy.

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader




# Define transformations to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

# Initialize the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Evaluation function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train and evaluate the model
epochs = 3
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322520
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.133983
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.193611
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.205039
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.128459
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.105999
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.059856
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.087469
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.198047
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.012094

Test set: Average loss: 0.0000, Accuracy: 9856/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.079127
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.013837
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.155971
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.026608
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.017832
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.025762
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.019373
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.018458
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.137485
T

In [32]:
import random
torch.save(model.state_dict(), "mnist_model.pth")
# Now, let's load the model state dictionary into a new instance of the model
loaded_model = Net()
loaded_model.load_state_dict(torch.load("mnist_model.pth"))
loaded_model.to(device)
loaded_model.eval() # Set the model to evaluation mode

# Example of using the loaded model to make a prediction with a random input
# Select a random index from the test dataset
random_index = random.randint(0, len(test_dataset) - 1)
sample_image, sample_label = test_dataset[random_index]

# Add a batch dimension to the image
sample_image = sample_image.unsqueeze(0)
sample_image = sample_image.to(device)

# Make a prediction
with torch.no_grad():
  output = loaded_model(sample_image)
  prediction = output.argmax(dim=1, keepdim=True)

print(f"Original Label: {sample_label}")
print(f"Predicted Label: {prediction.item()}")
print("\nEvaluating loaded model:")
test(loaded_model, device, test_loader)


Original Label: 1
Predicted Label: 1

Evaluating loaded model:

Test set: Average loss: 0.0000, Accuracy: 9870/10000 (99%)

